In [ ]:
# Get imports
import pandas as pd
import os

# Set path names
schools_path = os.path.join('Resources','schools_complete.csv')
students_path = os.path.join('Resources','students_complete.csv')

# Create dataframes
schools_df = pd.read_csv(schools_path, delimiter=',')
students_df = pd.read_csv(students_path, delimiter=',')

schools_df

In [ ]:
# Total number of schools in district is length of schools data frame
total_schools = len(schools_df)
# Total number of students in district is length of students data frame
total_students = len(students_df)
# Total budget for district is the sum of the budget column in schools data frame
total_budget = schools_df['budget'].sum()
# Average math = mean of student's math scores
average_math_score = round(students_df['math_score'].mean())
# Average reading = mean of student's reading scores
average_reading_score = round(students_df['reading_score'].mean())

# Create variable to control passing grade - default is higher than 60
passing = 69.99999

# Create a new boolean column in students data frame for if studennt is passing math 
students_df.loc[students_df['math_score'] > passing, 'passing_math'] = True
students_df.loc[students_df['math_score'] <= passing, 'passing_math'] = False
# Total number of students passing math is length of studnets data frame with passing math True
total_passing_math = len(students_df[students_df['passing_math'] == True])
# Percent passing math is total number passing math divided by total number of students
total_percent_math = round(100 * total_passing_math / total_students, 2)

# Create boolean column for passing reading
students_df.loc[students_df['reading_score'] > passing, 'passing_reading'] = True
students_df.loc[students_df['reading_score'] <= passing, 'passing_reading'] = False
# Total number of students passing reading is length of students data frame with passing reading Ture
total_passing_reading = len(students_df[students_df['passing_reading'] == True])
# Percent passing reading is total number passing reading / total number of students
total_percent_reading = round(100* total_passing_reading / total_students, 2)

# Create a 3rd boolean column for passing total = passing math and passing reading
students_df['passing_overall'] = False
students_df.loc[(students_df['passing_math'] == True) & (students_df['passing_reading'] == True), 'passing_overall'] = True
# Total number of students passing overall is length of students dataframe with passing overall True
total_passing_overall = len(students_df[students_df['passing_overall'] == True])
# Percent passing overall is total number passing overall / total number of students
total_percent_overall = round(100 * total_passing_overall / total_students, 2)

students_df

In [ ]:
# Create dataframe with results
district_df = pd.DataFrame({'Total Number of Schools':total_schools, 'Total Number of Students':"{:,}".format(total_students), 'Total Budget':f'${"{:,}".format(total_budget)}', 'Average Math Score':average_math_score, 'Average Reading Score':average_reading_score, '% Passing Math':total_percent_math, '% Passing Reading':total_percent_reading, '% Passing Overall':total_percent_overall},index=['District'])
district_df

In [ ]:
summary_df = schools_df[['type','size','budget']]
summary_df.set_axis(schools_df['school_name'], axis ='index', inplace=True)
summary_df.rename_axis('', inplace=True)
summary_df = summary_df.sort_index()